In [6]:
import numpy as np

# Generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size=20)

# Les valeurs importantes sont aux debut pour facilité le debuggage
params[0] = params.max() + 1
params[1] = params.min() -1
params[2] = 0

# arondi a la deuxieme decimal
params = np.round(params, 2)

# show
print(params)

[127.54 -46.75   0.    64.61 -30.12 -36.49  84.76 -45.75  72.6   23.35
 -37.24 126.54  34.03  85.34 106.85  -3.66 -12.48 101.93  74.2   37.73]


## Quantization method


In [12]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) ->  np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    
    # Quantize the parameters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantization(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

def symmetric_dequantization(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array,float]:
    
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha  / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale
    
def quantization_error(params: np.array, params_q: np.array):
 
    # Calculate the MSE
    return np.mean((params - params_q) **2)

(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[127.54 -46.75   0.    64.61 -30.12 -36.49  84.76 -45.75  72.6   23.35
 -37.24 126.54  34.03  85.34 106.85  -3.66 -12.48 101.93  74.2   37.73]

Asymmetric scale: 0.6834901960784314, zero: 68.0
[255   0  68 163  24  15 192   1 174 102  14 253 118 193 224  63  50 217
 177 123]

Symmetric scale: 1.004251968503937
[127 -47   0  64 -30 -36  84 -46  72  23 -37 126  34  85 106  -4 -12 101
  74  38]


In [15]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantization(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantization(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequntize Asymmetric: ')
print(np.round(params_deq_asymmetric, 2))
print('')
print(f'Dequantize Symmetric: ')
print(np.round(params_deq_asymmetric, 2))

Original:
[127.54 -46.75   0.    64.61 -30.12 -36.49  84.76 -45.75  72.6   23.35
 -37.24 126.54  34.03  85.34 106.85  -3.66 -12.48 101.93  74.2   37.73]

Dequntize Asymmetric: 
[127.81 -46.48   0.    64.93 -30.07 -36.22  84.75 -45.79  72.45  23.24
 -36.91 126.45  34.17  85.44 106.62  -3.42 -12.3  101.84  74.5   37.59]

Dequantize Symmetric: 
[127.81 -46.48   0.    64.93 -30.07 -36.22  84.75 -45.79  72.45  23.24
 -36.91 126.45  34.17  85.44 106.62  -3.42 -12.3  101.84  74.5   37.59]


In [16]:
# Quantization error

print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric erro: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.04
    Symmetric erro: 0.09
